### Import

In [ ]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.path.dirname("notebooks"), '..')))

from utils.func_preprocessing import (
    calculate_age,
    calculate_pf,
    calculate_podt,
    classify_cod_patology
)

from config.config import (
    DATASET_NAME,
    YEARS_TRAINING
)

Define columns to use

In [ ]:
columns_emogas = [
    'NCAMPIONE', 'ID', "NOME", "COGNOME", "DATA_NASCITA", 'NACCESSO', 'DATA',
    'TIPO', 'STATO', 'REPARTO_PZ', 'SESSO', 'ANNI', 'AADPO2', 'PF', 'PO2_T',
    'P50_ACT', 'TO2', 'SO2', 'PO2.1', 'HCT', 'AG_K', 'THB', 'THB2', 'GLU', 'LAC',
    'HCO3', 'PCO2_T', 'PCO2', 'MOSM', 'KP', 'NA', 'CL', 'CBASE', 'METHB', 'PHT',
    'PH', 'O2HB', 'COHB', 'RHB', 'B', 'TC', 'FIO2', 'P50_ST', 'RQ', 'P50', 'PEEP',
    'PS', 'CA'
    ]

columns_ps = [
    'ID_ANAGRAFICA', "NOME", "COGNOME", 'DTN', 'SESSO', 'DATA_INGRESSO',
    'DATA_USCITA', 'COD_PATOLOGIA', 'COD_DIAGNOSI_PRINCIPALE'
    ]

### Read data

In [ ]:
# Initialize an empty list to store DataFrames
df_emogas_list = []

# Loop through each year, read the corresponding Excel file, and append to the list
for year in YEARS_TRAINING:
    file_path = f"../raw_dataset/Import_Emogas_{year}.xlsx"
    df_year = pd.read_excel(file_path)[columns_emogas]
    # Rename 'PO2.1' to 'PO2' if it exists
    if 'PO2.1' in df_year.columns:
        df_year = df_year.rename(columns={'PO2.1': 'PO2'})
    df_emogas_list.append(df_year)

# Concatenate all DataFrames into a single DataFrame
df_emogas = pd.concat(df_emogas_list, ignore_index=True)

In [ ]:
# Initialize an empty list to store DataFrames
df_ps_list = []

# Loop through each year, read the corresponding Excel file, and append to the list
for year in YEARS_TRAINING:
    file_path = f"../raw_dataset/RIEP_{year}.xlsx"
    df_year = pd.read_excel(file_path)[columns_ps]
    df_ps_list.append(df_year)

# Concatenate all DataFrames into a single DataFrame
df_ps = pd.concat(df_ps_list, ignore_index=True)


### Preprocessing

In [ ]:
# Create a copy of the original DataFrame
df_temp_ps = df_ps.copy()

# Extract components from 'COD_DIAGNOSI_PRINCIPALE'
df_temp_ps['lettera_diagnosi_princip'] = df_temp_ps['COD_DIAGNOSI_PRINCIPALE'].str.extract(r'([A-Za-z]+)', expand=False)

# Extract the integer part of the diagnosis code
df_temp_ps['intero_diagnosi_princip'] = df_temp_ps['COD_DIAGNOSI_PRINCIPALE'].str.extract(r'(\d+)\.', expand=False)
df_temp_ps['intero_diagnosi_princip'] = df_temp_ps['intero_diagnosi_princip'].fillna(
    df_temp_ps['COD_DIAGNOSI_PRINCIPALE'].str.extract(r'(\d+)', expand=False)
)

# Extract the decimal part of the diagnosis code
df_temp_ps['decimali_diagnosi_princip'] = df_temp_ps['COD_DIAGNOSI_PRINCIPALE'].str.extract(r'\.(\d+)', expand=False)
df_temp_ps['decimali_diagnosi_princip'] = df_temp_ps['decimali_diagnosi_princip'].fillna(
    df_temp_ps['COD_DIAGNOSI_PRINCIPALE'].apply(lambda x: '0' if pd.notna(x) and '.' not in str(x) else None)
)

# Calculate age using a helper function
df_temp_ps['ETA'] = df_temp_ps.apply(
    lambda row: calculate_age(row['DTN'], row['DATA_INGRESSO']), axis=1
)

# Select relevant columns
columns_to_keep = [
    'ID_ANAGRAFICA', "NOME", "COGNOME", 'DTN', 'SESSO', 'ETA', 
    'DATA_INGRESSO', 'DATA_USCITA', 'COD_PATOLOGIA', 'COD_DIAGNOSI_PRINCIPALE', 
    'lettera_diagnosi_princip', 'intero_diagnosi_princip', 'decimali_diagnosi_princip'
]
df_temp_ps = df_temp_ps[columns_to_keep]

# Filter out rows with null 'COD_DIAGNOSI_PRINCIPALE'
df_temp_ps = df_temp_ps.loc[df_temp_ps['COD_DIAGNOSI_PRINCIPALE'].notnull()]

# Fill missing 'lettera_diagnosi_princip' values with "N"
df_temp_ps['lettera_diagnosi_princip'] = df_temp_ps['lettera_diagnosi_princip'].fillna("N")

In [ ]:
# Preprocess the Emogas dataset
df_temp_emogas = df_emogas[
    [
        'NCAMPIONE', 'ID', 'NOME', 'COGNOME', 'DATA_NASCITA', 'NACCESSO', 'DATA',
        'TIPO', 'STATO', 'REPARTO_PZ', 'SESSO', 'ANNI',
        'AADPO2', 'PF', 'PO2_T', 'P50_ACT', 'TO2', 'SO2', 'PO2',  # PO2 added, SO21 replaced with SO2
        'HCT', 'AG_K', 'THB', 'THB2', 'GLU', 'LAC', 'HCO3', 'PCO2_T', 'PCO2', 
        'MOSM', 'KP', 'NA', 'CL', 'CBASE', 'METHB', 'PHT', 'PH', 'O2HB', 'COHB', 
        'RHB', 'B', 'TC', 'FIO2', 'P50_ST', 'RQ', 'P50', 'PEEP', 'PS', 'CA'
    ]
].rename(
    columns={"ID": "ID_ANAGRAFICA", "DATA_NASCITA": "DTN"}
)

# Filter rows with valid STATO values
df_temp_emogas = df_temp_emogas[df_temp_emogas['STATO'].isin(['?', 'OK'])]

# Drop rows with missing critical fields
required_fields = ['NOME', 'COGNOME', 'ID_ANAGRAFICA']
df_temp_emogas = df_temp_emogas.dropna(subset=required_fields)

In [ ]:
# Convert data types for ID_ANAGRAFICA to string
df_temp_ps['ID_ANAGRAFICA'] = df_temp_ps['ID_ANAGRAFICA'].astype(str)
df_temp_emogas['ID_ANAGRAFICA'] = df_temp_emogas['ID_ANAGRAFICA'].astype(str)

# Convert date columns to datetime format in df_temp_ps
date_columns_ps = ['DATA_INGRESSO', 'DATA_USCITA', 'DTN']
df_temp_ps[date_columns_ps] = df_temp_ps[date_columns_ps].apply(pd.to_datetime)

# Convert date column to datetime format in df_temp_emogas
df_temp_emogas['DATA'] = pd.to_datetime(df_temp_emogas['DATA'])


### Join

In [ ]:
# Merge df_temp_ps and df_temp_emogas on 'ID_ANAGRAFICA'
merged_df_id = pd.merge(
    df_temp_ps,
    df_temp_emogas.drop(columns=["NOME", "COGNOME", "DTN", "SESSO"]),
    on='ID_ANAGRAFICA',
    how='inner'
)

# Filter rows where 'DATA' falls within the range of 'DATA_INGRESSO' and 'DATA_USCITA'
result_id = merged_df_id[
    (merged_df_id['DATA'] >= merged_df_id['DATA_INGRESSO']) &
    (merged_df_id['DATA'] <= merged_df_id['DATA_USCITA'])
]

# Sort values by 'ID_ANAGRAFICA', 'DATA_INGRESSO', and 'DATA'
result_id = result_id.sort_values(by=["ID_ANAGRAFICA", "DATA_INGRESSO", "DATA"])

# Remove duplicates, keeping the first occurrence for each 'ID_ANAGRAFICA' and 'DATA_INGRESSO'
result_id = result_id.drop_duplicates(subset=["ID_ANAGRAFICA", "DATA_INGRESSO"], keep='first')


In [ ]:
# Merge df_temp_ps and df_temp_emogas on 'NOME' and 'COGNOME'
merged_df_name = pd.merge(
    df_temp_ps,
    df_temp_emogas.drop(columns=["ID_ANAGRAFICA", "DTN", "SESSO"]),
    on=["NOME", "COGNOME"],
    how='inner'
)

# Filter rows where 'DATA' falls within the range of 'DATA_INGRESSO' and 'DATA_USCITA'
result_name = merged_df_name[
    (merged_df_name['DATA'] >= merged_df_name['DATA_INGRESSO']) &
    (merged_df_name['DATA'] <= merged_df_name['DATA_USCITA'])
]

# Sort values by 'NOME', 'COGNOME', 'DATA_INGRESSO', and 'DATA'
result_name = result_name.sort_values(by=["NOME", "COGNOME", "DATA_INGRESSO", "DATA"])

# Remove duplicates, keeping the first occurrence for each 'NOME', 'COGNOME', and 'DATA_INGRESSO'
result_name = result_name.drop_duplicates(subset=["NOME", "COGNOME", "DATA_INGRESSO"], keep='first')

# Concatenate result_id and result_name DataFrames
concat = pd.concat([result_id, result_name])

# Drop duplicate rows across both merges
df_final = concat.drop_duplicates()


### Cleaning

In [ ]:
# Select and reorder columns in the final DataFrame
df_final = df_final[
    [
        'ID_ANAGRAFICA', 'DTN', 'SESSO', 'ETA', 
        'DATA_INGRESSO', 'DATA', 'DATA_USCITA', 
        'NCAMPIONE', 'NACCESSO', 'TIPO', 'STATO', 'REPARTO_PZ',
        'COD_PATOLOGIA', 'AADPO2', 'PF', 'PO2_T', 'P50_ACT',
        'TO2', 'SO2', 'HCT', 'AG_K', 'THB', 'THB2', 'GLU', 
        'LAC', 'PO2',  # Removed PO21 and replaced SO21 with SO2
        'HCO3', 'PCO2_T', 'PCO2', 'MOSM', 'KP', 'NA', 'CL', 
        'CBASE', 'METHB', 'PHT', 'PH', 'O2HB', 'COHB', 
        'RHB', 'B', 'TC', 'FIO2', 'P50_ST', 'RQ', 'P50', 
        'PEEP', 'PS', 'CA',  # Added CA
        'COD_DIAGNOSI_PRINCIPALE', 'lettera_diagnosi_princip', 
        'intero_diagnosi_princip', 'decimali_diagnosi_princip'
    ]
]

In [ ]:
# Calculate 'PO2_T_calc' based on 'PO2' and 'TC', and update 'PO2_T' if missing
df_final["PO2_T_calc"] = df_final.apply(
    lambda row: calculate_podt(row['PO2'], row['TC']) 
    if pd.notnull(row['PO2']) and pd.notnull(row['TC']) and row['TC'] != 0 and row['PO2'] != '.....' 
    else None, 
    axis=1
)

# Fill missing values in 'PO2_T' with 'PO2_T_calc' and drop the temporary column
df_final['PO2_T'] = df_final['PO2_T'].fillna(df_final['PO2_T_calc'])
df_final = df_final.drop(columns=["PO2_T_calc"])

# Calculate 'PF' based on 'PO2_T' and 'FIO2'
df_final["PF"] = df_final.apply(
    lambda row: calculate_pf(row['PO2_T'], row['FIO2']) 
    if pd.notnull(row['PO2_T']) and pd.notnull(row['FIO2']) and row['FIO2'] != 0 
    else None, 
    axis=1
)

In [ ]:
# Create a copy of the original DataFrame
cleaned_df = df_final.copy()

# Initialize a DataFrame to collect "dirty" rows and their reasons
dirty_df = pd.DataFrame()

# Helper function to mark and collect dirty rows
def collect_dirty_rows(df, condition, reason):
    tmp_dirty = df.loc[condition].copy()
    tmp_dirty["Motivo"] = reason
    return tmp_dirty

# 1. Handle `ETA` (Age)
dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, cleaned_df.ETA == 0, "ETA uguale a 0")])
cleaned_df = cleaned_df.loc[cleaned_df.ETA > 0]

# 2. Handle `GLU`
dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, cleaned_df.GLU == '.....', "GLU == .....")])
cleaned_df = cleaned_df.loc[cleaned_df.GLU != '.....']

cleaned_df["GLU"] = cleaned_df["GLU"].astype(float)
dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, cleaned_df.GLU <= 6, "GLU minore di 6")])
dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, cleaned_df.GLU.isnull(), "GLU nullo")])
cleaned_df = cleaned_df.loc[cleaned_df.GLU > 6]

# 3. Handle `PCO2_T`
dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, cleaned_df.PCO2_T < 10, "PCO2_T minore di 10")])
dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, cleaned_df.PCO2_T.isnull(), "PCO2_T nullo")])
cleaned_df = cleaned_df.loc[cleaned_df.PCO2_T > 10]

# 4. Handle `TC`
dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, cleaned_df.TC < 33, "TC minore di 33")])
cleaned_df = cleaned_df.loc[cleaned_df.TC >= 33]

# 5. Handle `FIO2`
dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, cleaned_df.FIO2 < 21, "FIO2 minore di 21")])
cleaned_df = cleaned_df.loc[cleaned_df.FIO2 >= 21]

# 6. Handle `TIPO`
dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, ~cleaned_df.TIPO.isin(["Arterioso", "Venoso"]), "TIPO inaspettato")])
cleaned_df = cleaned_df.loc[cleaned_df.TIPO.isin(["Arterioso", "Venoso"])]

# 7. Handle `THB`
dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, cleaned_df.THB == '.....', "THB == .....")])
cleaned_df = cleaned_df.loc[cleaned_df.THB != '.....']

dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, cleaned_df.THB < 3, "THB minore di 3")])
dirty_df = pd.concat([dirty_df, collect_dirty_rows(cleaned_df, cleaned_df.THB.isnull(), "THB nullo")])
cleaned_df = cleaned_df.loc[cleaned_df.THB >= 3]


In [ ]:
# Drop columns with uniform values
columns_to_drop_uniform = ["RQ", "P50", "P50_ST"]
cleaned_df = cleaned_df.drop(columns=columns_to_drop_uniform)

# Drop columns with too many null values
columns_to_drop_nulls = ["PEEP", "PS", "AADPO2"]
cleaned_df = cleaned_df.drop(columns=columns_to_drop_nulls)

In [ ]:
# Remove rows with any null values
cleaned_df = cleaned_df.dropna()

# Collect rows with null values and record the reason
tmp_dirty = cleaned_df[cleaned_df.isna().any(axis=1)].copy()
tmp_dirty["Motivo"] = "Qualche colonna nulla"

# Append the dirty rows to the dirty_df
dirty_df = pd.concat([dirty_df, tmp_dirty], ignore_index=True)

In [ ]:
print(cleaned_df.shape)
print(dirty_df.shape)

In [ ]:
# Identify rows containing the string '.....' in any column
tmp_dirty = cleaned_df[
    cleaned_df.apply(lambda row: row.astype(str).str.contains(r'\.\.\.\.\.', regex=True)).any(axis=1)
].copy()

# Add a 'Motivo' column specifying the reason for exclusion
tmp_dirty["Motivo"] = "Qualche colonna contiene ....."

# Append the identified rows to the dirty_df
dirty_df = pd.concat([dirty_df, tmp_dirty], ignore_index=True)

# Remove rows containing '.....' from the original DataFrame
cleaned_df = cleaned_df[
    ~cleaned_df.apply(lambda row: row.astype(str).str.contains(r'\.\.\.\.\.', regex=True)).any(axis=1)
]


In [ ]:
print(cleaned_df.shape)
print(dirty_df.shape)

In [ ]:
# Convert 'COD_PATOLOGIA' column to integer
cleaned_df['COD_PATOLOGIA'] = cleaned_df['COD_PATOLOGIA'].astype(int)

# Apply classification function to 'COD_PATOLOGIA' and create a new column 'class_symptom'
cleaned_df['class_symptom'] = cleaned_df['COD_PATOLOGIA'].apply(classify_cod_patology)

# Drop the 'COD_PATOLOGIA' column after classification
cleaned_df = cleaned_df.drop(columns=['COD_PATOLOGIA'])


In [ ]:
print(cleaned_df.shape)
print(dirty_df.shape)

### Save

In [ ]:
# Save the cleaned DataFrame to a CSV file
output_path = f"../dataset/{DATASET_NAME}.csv"
cleaned_df.to_csv(output_path, index=False)  # Save without the index for a cleaner output